In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/roohankhan/housing-pakistan/train.csv
/kaggle/input/d/roohankhan/housing-pakistan/test.csv


## PRICE PREDICTOR FOR HOUSES IN PAKISTAN

PREPROCESSING DATA

In [2]:
from sklearn.model_selection import train_test_split

# Path of the file to read
X_train_path = '../input/d/roohankhan/housing-pakistan/train.csv'
X_test_path = '../input/d/roohankhan/housing-pakistan/test.csv'

# CSV to DF
X_full = pd.read_csv(X_train_path, index_col='property_id')
X_test_full = pd.read_csv(X_test_path, index_col='property_id')

# Drop extra columns
X_test_full.drop(['price'], axis=1, inplace=True)

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['price'], inplace=True)
y = X_full.price
X_full.drop(['price'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

CHECK FOR MISSING VALUES

In [3]:
X_full.isnull().sum()

property_type    0
location         0
city             0
province_name    0
latitude         0
longitude        0
baths            0
area             0
bedrooms         0
Area Type        0
Area Size        0
Area Category    0
dtype: int64

IMPORT FUNCTIONS TO BE USED

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

FUNCTION FOR COMPARING DIFFERENT RESOURCES

In [5]:
# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

DROPPING CATEGORICAL COLUMNS

In [6]:
X_train = X_train_full.copy()
X_valid = X_valid_full.copy()

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop categorical variables):
8633843.919376988


ORDINAL ENCODING

In [7]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

#print('Categorical columns that will be ordinal encoded:', good_label_cols)
#print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

# Apply ordinal encoder 
ordinal_encoder=OrdinalEncoder()
label_X_train[good_label_cols]=ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols]=ordinal_encoder.transform(X_valid[good_label_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
7573727.600066238


ONE-HOT ENCODING

In [8]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

#print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
#print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

# Use as many lines of code as you need!
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
7540192.860340426


SEPERATING NUMERICAL AND CATEGORICAL COLUMNS

In [9]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

USING A PIPELINE

In [10]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 7540192.860340426


In [11]:
#preds = clf.predict(X_test)
#print(preds)

# Save test predictions to file
#output = pd.DataFrame({'Id': X_test.index,'predicted_price': preds})
#output.to_csv('submission_4.csv', index=False)

USING CROSS VALIDATION

In [12]:
# Multiply by -1 since sklearn calculates *negative* MAE
#scores = -1 * cross_val_score(clf, X_full, y ,cv=5, scoring='neg_mean_absolute_error')

#print("MAE scores: ", scores)
#print("Average MAE score (across experiments):")
#print(scores.mean())

PREPROCESSING DATA FOR XGBOOSTER

In [13]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train_XG = pd.get_dummies(X_train)
X_valid_XG = pd.get_dummies(X_valid)
X_test_XG = pd.get_dummies(X_test)
X_train_XG, X_valid_XG = X_train_XG.align(X_valid_XG, join='left', axis=1)
X_train_XG, X_test_XG = X_train_XG.align(X_test_XG, join='left', axis=1)

WITHOUT PARAMETER TUNING

In [14]:
# Define the model
my_model_1 = XGBRegressor(random_state=0)

# Fit the model
my_model_1.fit(X_train_XG,y_train)

# Get predictions
predictions_1 = my_model_1.predict(X_valid_XG)

# Calculate MAE
mae_1 = mean_absolute_error(y_valid, predictions_1)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 8749547.450715035


WITH PARAMETER TUNING

In [15]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=500, n_jobs=4, early_stopping_rounds=5, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train_XG, y_train, eval_set=[(X_valid_XG, y_valid)], verbose=False)

# Get predictions
predictions_2 = my_model_2.predict(X_valid_XG)

# Calculate MAE
mae_2 = mean_absolute_error(y_valid, predictions_2)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 8818791.632414185


FINAL OUTPUT